In [1]:
import os
import pandas_profiling as pp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

import datetime
import lightgbm as lgb
import pandas_profiling as pp
from scipy import stats
from scipy.sparse import hstack, csr_matrix, vstack
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
stop = set(stopwords.words('russian'))
import string
from collections import Counter
import re
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
#import utils
import pickle
from sklearn.linear_model import LinearRegression
%load_ext autoreload
%autoreload 2

pd.set_option('max_columns', 150)

import glob
from dataprocesser import DataProcesser
from features import MetaFeaturesGenerator
#from utils import DataProcesser
import gc
from scipy.stats import boxcox

In [2]:
def rmse(predictions, targets):
    return np.sqrt(((np.clip(predictions, 0, 1) - targets) ** 2).mean())

In [3]:
price_log_tree_usual_train = np.load('f:/Avito/data/price_log_tree_usual_train.npy')
price_log_tree_usual_test = np.load('f:/Avito/data/price_log_tree_usual_test.npy')
other_columns_usual_train = np.load('f:/Avito/data/other_columns_usual_train.npy')
other_columns_usual_test = np.load('f:/Avito/data/other_columns_usual_test.npy')
cat_features_me_usual_train100_20_001 = np.load('f:/Avito/data/cat_features_me_usual_train100_20_0.01.npy')
cat_features_me_usual_test100_20_001 = np.load('f:/Avito/data/cat_features_me_usual_test100_20_0.01.npy')

In [4]:
train_ = pd.read_csv('f:/avito/train.csv', usecols = ['item_seq_number', 'price'])
item_seq_number_train = train_['item_seq_number']
test_ = pd.read_csv('f:/avito/test.csv', usecols = ['item_seq_number', 'price'])
item_seq_number_test = test_['item_seq_number']
del train_
del test_

In [5]:
# reading meta-features from selected folder.
X = None
train_files = [i for i in glob.glob('pickles/1_5_sublinear_tf-True_smooth_idf-False/*') if 'X_meta' in i]
for i, file in enumerate(train_files):
    with open(file, 'rb') as f:
        X_temp = pickle.load(f)
    X = X_temp if X is None else np.append(X, X_temp, axis=1)
    del X_temp
    
X_test = None
test_files = [i for i in glob.glob('pickles/1_5_sublinear_tf-True_smooth_idf-False/*') if 'X_test_meta' in i]
for i, file in enumerate(test_files):
    with open(file, 'rb') as f:
        X_temp = pickle.load(f)
    X_temp = X_temp.reshape(-1, 1)
    X_test = X_temp if X_test is None else np.append(X_test, X_temp, axis=1)
    del X_temp

In [6]:
with open('train_description.pickle', 'rb') as f:
    train_description = pickle.load(f)
with open('test_description.pickle', 'rb') as f:
    test_description = pickle.load(f)
    
with open('model_active.pickle', 'rb') as f:
    model_active = pickle.load(f)
    
X_meta_active = model_active.predict(train_description)
X_test_meta_active = model_active.predict(test_description)

In [7]:
X_meta_active = np.log1p(X_meta_active)
X_test_meta_active = np.log1p(X_test_meta_active)


d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in log1p
  if __name__ == '__main__':
d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in log1p
  from ipykernel import kernelapp as app


In [8]:
d = pd.read_csv('train.csv')

In [18]:
interesting_columns = ["region", "city", "parent_category_name", "category_name", "param_1", 'price', 'image', 'item_id']
train = pd.read_csv('train.csv', usecols = interesting_columns)
test = pd.read_csv('test.csv', usecols = interesting_columns)

In [19]:
df = pd.concat([train, test], axis=0)

In [20]:
df['param_1'] = df['param_1'].fillna('missing')
df['region_city'] = df['region'] + '_' + df['city']
df['parent_category_name_category_name'] = df['parent_category_name'] + ' ' + df['category_name']
df['parent_category_name_param_1'] = df['parent_category_name'] + ' ' + df['param_1']
df['category_name_param1'] = df['category_name'] + ' ' + df['param_1']

In [21]:
for col in ['region_city', 'parent_category_name_category_name', 'parent_category_name_param_1', 'category_name_param1']:
    lbl = LabelEncoder()
    df[col] = lbl.fit_transform(df[col].values.astype('str'))

In [22]:
train_img = pd.read_csv('train_img_features_v1.csv')
test_img = pd.read_csv('test_img_features_v1.csv')

In [24]:
train = df[:1503424]
test = df[1503424:]

In [25]:
train['image'] = train['image'].astype('category')
train_img['image'] = train_img['image'].astype('category')
train['item_id'] = train['item_id'].astype('category')
train_img['item_id'] = train_img['item_id'].astype('category')

d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [26]:
train = pd.merge(train, train_img, on='item_id', how='left')
test = pd.merge(test, test_img, on='item_id', how='left')

In [27]:
train['image_x'] = train['image_x'].cat.add_categories([0])
train['image_y'] = train['image_y'].cat.add_categories([0])

train['image_x'] = train['image_x'].fillna(0)
test['image_x'] = test['image_x'].fillna(0)
train['image_y'] = train['image_y'].fillna(0)
test['image_y'] = test['image_y'].fillna(0)

In [28]:
for col in ['img_mean_color', 'img_std_color', 'img_blue_mean', 'img_green_mean',
       'img_red_mean', 'img_blue_std', 'image_green_std', 'image_red_std', 'img_size_x', 'img_size_y']:
    train[col] = train[col].fillna(0)
    test[col] = test[col].fillna(0)

In [29]:
y = d['deal_probability']
del d

In [22]:
with open('pickles/1_5_sublinear_tf-True_smooth_idf-False/X_texts.pickle', 'rb') as f:
    X_texts = pickle.load(f)
    
with open('pickles/1_5_sublinear_tf-True_smooth_idf-False/X_test_texts.pickle', 'rb') as f:
    X_test_texts = pickle.load(f)


In [9]:
resnet_scores_train = pd.read_csv('resnet_scores_train.csv')
resnet_scores_test = pd.read_csv('resnet_scores_test.csv')

In [10]:
resnet_scores_train.fillna(0, inplace=True)
resnet_scores_test.fillna(0, inplace=True)

In [11]:
nasnet_scores_train_fixed = pd.read_csv('nasnet_scores_train_fixed.csv')
nasnet_scores_test_fixed = pd.read_csv('nasnet_scores_test_fixed.csv')

In [12]:
mobilenet_scores_train_fixed = pd.read_csv('mobilenet_scores_train_fixed.csv')
mobilenet_scores_test_fixed = pd.read_csv('mobilenet_scores_test_fixed.csv')

In [13]:
nasnet_scores_train_fixed.fillna(0, inplace=True)
nasnet_scores_test_fixed.fillna(0, inplace=True)
mobilenet_scores_train_fixed.fillna(0, inplace=True)
mobilenet_scores_test_fixed.fillna(0, inplace=True)

In [14]:
resnet_features = ['resnet_nima_feature_0',
       'resnet_nima_feature_1', 'resnet_nima_feature_2',
       'resnet_nima_feature_3', 'resnet_nima_feature_4',
       'resnet_nima_feature_5', 'resnet_nima_feature_6',
       'resnet_nima_feature_7', 'resnet_nima_feature_8',
       'resnet_nima_feature_9']
nasnet_features = ['nasnet_nima_feature_0',
       'nasnet_nima_feature_1', 'nasnet_nima_feature_2',
       'nasnet_nima_feature_3', 'nasnet_nima_feature_4',
       'nasnet_nima_feature_5', 'nasnet_nima_feature_6',
       'nasnet_nima_feature_7', 'nasnet_nima_feature_8',
       'nasnet_nima_feature_9']
mobilenet_features = ['MobileNet_nima_feature_0', 'MobileNet_nima_feature_1',
       'MobileNet_nima_feature_2', 'MobileNet_nima_feature_3',
       'MobileNet_nima_feature_4', 'MobileNet_nima_feature_5',
       'MobileNet_nima_feature_6', 'MobileNet_nima_feature_7',
       'MobileNet_nima_feature_8', 'MobileNet_nima_feature_9']

In [15]:
mobilenet_train_mean = mobilenet_scores_train_fixed[mobilenet_features].mean(axis=1).values.reshape(-1, 1)
mobilenet_train_std = mobilenet_scores_train_fixed[mobilenet_features].std(axis=1).values.reshape(-1, 1)
mobilenet_test_mean = mobilenet_scores_test_fixed[mobilenet_features].mean(axis=1).values.reshape(-1, 1)
mobilenet_test_std = mobilenet_scores_test_fixed[mobilenet_features].std(axis=1).values.reshape(-1, 1)

In [51]:
#%%time
X_meta_ = hstack([X_texts, np.log1p(X_meta_active.reshape(-1, 1)), price_log_tree_usual_train.reshape(-1, 1),
                     other_columns_usual_train[:, 1:], item_seq_number_train.values.reshape(-1, 1),
                    cat_features_me_usual_train100_20_001[:, :8],
                    train[['img_size_x', 'img_size_y',
       'img_mean_color', 'img_std_color', 'img_blue_mean', 'img_green_mean',
       'img_red_mean', 'img_blue_std', 'image_green_std', 'image_red_std']].values,
                nima_train[resnet_features].values])
X_test_full = hstack([X_test_texts, np.log1p(X_test_meta_active.reshape(-1, 1)), price_log_tree_usual_test.reshape(-1, 1),
                         other_columns_usual_test[:, 1:],
                         item_seq_number_test.values.reshape(-1, 1),
                         cat_features_me_usual_test100_20_001[:, :8],
                    test[['img_size_x', 'img_size_y',
       'img_mean_color', 'img_std_color', 'img_blue_mean', 'img_green_mean',
       'img_red_mean', 'img_blue_std', 'image_green_std', 'image_red_std']].values,
                nima_test[resnet_features].values])
X_train, X_valid, y_train, y_valid = train_test_split(X_meta_, y, test_size=0.20, random_state=42)
params = {'learning_rate': 0.05, 'max_depth': 13, 'boosting': 'gbdt', 'objective': 'regression',
          'metric': ['logloss', 'rmse'],
          'is_training_metric': True, 'seed': 19, 'num_leaves': 256, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
          'bagging_freq': 5, 'lambda_l1': 4, 'lambda_l2': 4, 'num_threads': 12}

model5 = lgb.train(params, lgb.Dataset(X_train.tocsr(), label=y_train),10000,
                           lgb.Dataset(X_valid.tocsr(), label=y_valid),
                           verbose_eval=100, early_stopping_rounds=100)
print(rmse(model5.predict(X_valid.tocsr()), y_valid))


# [4200]	valid_0's rmse: 0.21661
# Early stopping, best iteration is:
# [4130]	valid_0's rmse: 0.216604
# 0.2165386270936107

d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in log1p
  from ipykernel import kernelapp as app
d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in log1p


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.222201
[200]	valid_0's rmse: 0.221099
[300]	valid_0's rmse: 0.220401
[400]	valid_0's rmse: 0.219855
[500]	valid_0's rmse: 0.219528
[600]	valid_0's rmse: 0.21918
[700]	valid_0's rmse: 0.218913
[800]	valid_0's rmse: 0.218702
[900]	valid_0's rmse: 0.218475
[1000]	valid_0's rmse: 0.218295
[1100]	valid_0's rmse: 0.218172
[1200]	valid_0's rmse: 0.218059
[1300]	valid_0's rmse: 0.217937
[1400]	valid_0's rmse: 0.217794
[1500]	valid_0's rmse: 0.217691
[1600]	valid_0's rmse: 0.217581
[1700]	valid_0's rmse: 0.217493
[1800]	valid_0's rmse: 0.217381
[1900]	valid_0's rmse: 0.2173
[2000]	valid_0's rmse: 0.217222
[2100]	valid_0's rmse: 0.217167
[2200]	valid_0's rmse: 0.217078
[2300]	valid_0's rmse: 0.217045
[2400]	valid_0's rmse: 0.216998
[2500]	valid_0's rmse: 0.216963
[2600]	valid_0's rmse: 0.216936
[2700]	valid_0's rmse: 0.216889
[2800]	valid_0's rmse: 0.216861
[2900]	valid_0's rmse: 0.21684
[3000]	valid_0's rmse:

In [30]:
#%%time
X_meta_ = np.hstack([X, np.log1p(X_meta_active.reshape(-1, 1)), price_log_tree_usual_train.reshape(-1, 1),
                     other_columns_usual_train[:, 1:], item_seq_number_train.values.reshape(-1, 1),
                    cat_features_me_usual_train100_20_001[:, :8],
                    train[['img_size_x', 'img_size_y',
       'img_mean_color', 'img_std_color', 'img_blue_mean', 'img_green_mean',
       'img_red_mean', 'img_blue_std', 'image_green_std', 'image_red_std']].values,
                resnet_scores_train[resnet_features].values,
                nasnet_scores_train_fixed[nasnet_features].values,
                 mobilenet_train_mean, mobilenet_train_std])

X_test_full = np.hstack([X_test, np.log1p(X_test_meta_active.reshape(-1, 1)), price_log_tree_usual_test.reshape(-1, 1),
                         other_columns_usual_test[:, 1:],
                         item_seq_number_test.values.reshape(-1, 1),
                         cat_features_me_usual_test100_20_001[:, :8],
                    test[['img_size_x', 'img_size_y',
       'img_mean_color', 'img_std_color', 'img_blue_mean', 'img_green_mean',
       'img_red_mean', 'img_blue_std', 'image_green_std', 'image_red_std']].values,
                resnet_scores_test[resnet_features].values,
                nasnet_scores_test_fixed[nasnet_features].values,
                     mobilenet_test_mean, mobilenet_test_std])
X_train, X_valid, y_train, y_valid = train_test_split(X_meta_, y, test_size=0.20, random_state=42)
params = {'learning_rate': 0.05, 'max_depth': 12, 'boosting': 'gbdt', 'objective': 'regression',
          'metric': ['logloss', 'rmse'],
          'is_training_metric': True, 'seed': 19, 'num_leaves': 256, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
          'bagging_freq': 5, 'lambda_l1': 4, 'lambda_l2': 4, 'num_threads': 12}




# [4200]	valid_0's rmse: 0.21661
# Early stopping, best iteration is:
# [4130]	valid_0's rmse: 0.216604
# 0.2165386270936107

d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in log1p
  from ipykernel import kernelapp as app
d:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: invalid value encountered in log1p


In [32]:

model = lgb.train(params, lgb.Dataset(X_train, label=y_train),10000,
                           lgb.Dataset(X_valid, label=y_valid),
                           verbose_eval=100, early_stopping_rounds=100)
print(rmse(model.predict(X_valid, y_valid))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.221072
[200]	valid_0's rmse: 0.219715
[300]	valid_0's rmse: 0.219142
[400]	valid_0's rmse: 0.218802
[500]	valid_0's rmse: 0.218669
[600]	valid_0's rmse: 0.218504
[700]	valid_0's rmse: 0.218416
[800]	valid_0's rmse: 0.218326
[900]	valid_0's rmse: 0.218282
[1000]	valid_0's rmse: 0.218272
Early stopping, best iteration is:
[951]	valid_0's rmse: 0.218262


AttributeError: 'numpy.ndarray' object has no attribute 'tocsr'

In [36]:
rmse(model.predict(X_valid), y_valid)

0.21825336904470016

In [39]:
kf = KFold(n_splits=5)
X_meta = np.zeros((1503424, 1))
X_test_meta = []
for fold_i, (train_i, test_i) in enumerate(kf.split(X_meta_)):
    clf = model


    model = lgb.train(params, lgb.Dataset(X_meta_[train_i], label=y[train_i]),10000,
                       lgb.Dataset(X_meta_[test_i], label=y[test_i]),
                       verbose_eval=100, early_stopping_rounds=100)


    X_meta[test_i, :] = clf.predict(X_meta_[test_i]).reshape(-1, 1)
    X_test_meta.append(clf.predict(X_test_full))
X_test_meta = np.stack(X_test_meta)
X_test_meta_mean = np.mean(X_test_meta, axis=0)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.221172
[200]	valid_0's rmse: 0.219827
[300]	valid_0's rmse: 0.21929
[400]	valid_0's rmse: 0.218955
[500]	valid_0's rmse: 0.218826
[600]	valid_0's rmse: 0.218724
[700]	valid_0's rmse: 0.218638
[800]	valid_0's rmse: 0.218586
[900]	valid_0's rmse: 0.21853
[1000]	valid_0's rmse: 0.218521
[1100]	valid_0's rmse: 0.218516
[1200]	valid_0's rmse: 0.218489
Early stopping, best iteration is:
[1193]	valid_0's rmse: 0.218482
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.221344
[200]	valid_0's rmse: 0.220074
[300]	valid_0's rmse: 0.219555
[400]	valid_0's rmse: 0.219239
[500]	valid_0's rmse: 0.219037
[600]	valid_0's rmse: 0.218916
[700]	valid_0's rmse: 0.218856
[800]	valid_0's rmse: 0.218813
[900]	valid_0's rmse: 0.218744
[1000]	valid_0's rmse: 0.218726
[1100]	valid_0's rmse: 0.218721
[1200]	valid_0's rmse: 0.218717
Early stopping, best iteration is:
[1119]	valid_0's rmse: 0

In [41]:
np.save('lgb_for_stack_train.npy', X_meta)
np.save('lgb_for_stack_test.npy', X_test_meta)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['deal_probability'] = np.clip(pred, prediction, 1)
sub.to_csv('artgor.csv', index=False)

In [ ]:
print(rmse(model5.predict(X_valid), y_valid))

In [ ]:
pred = model5.predict(X_test_full.tocsr()).reshape(-1, 1)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['deal_probability'] = np.clip(pred, 0, 1)
sub.to_csv('artgor_single_lgb__image_other2.csv', index=False)

In [56]:
np.mean(pred)

0.15013417811652316

In [60]:
s1 = pd.read_csv('f:/Avito/subs/nikita_solo_2220.csv')
s2 = pd.read_csv('f:/Avito/subs/insaf_solo_2213.csv')
s3 = pd.read_csv('f:/Avito/subs/michael_solo_2229.csv')
s4 = pd.read_csv('f:/Avito/subs/andrew_solo_2228.csv')
s5 = pd.read_csv('f:/Avito/subs/lgsub.csv')
s6 = pd.read_csv('f:/Avito/subs/Avito_Shanth_RNN_AVERAGE.csv')

In [61]:
sub['deal_probability'] = np.clip((s1['deal_probability'] + s2['deal_probability'] + s3['deal_probability'] + s4['deal_probability'] + s5['deal_probability'] + s6['deal_probability'] + sub['deal_probability']) / 7, 0, 1)

(blend['N_solo'] * 0.20) + (blend['I_solo'] * 0.20) + (blend['M_solo'] * 0.20) + (blend['A_solo'] * 0.20) + (blend['K_Ridge'] * 0.10) + (blend['K_RNN'] * 0.10)

In [62]:
sub.to_csv('blend_artgor.csv', index=False)

In [63]:
np.mean(sub['deal_probability'])

0.15128793653127823

In [32]:
np.save('X_train_xgb.npy', X_meta_)
np.save('X_test_xgb.npy', X_test_full)

In [4]:
d = pd.read_csv('f:/Avito/train.csv')
y = d['deal_probability']
del d
X_meta_ = np.load('X_train_xgb.npy')
X_test_full = np.load('X_test_xgb.npy')

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_meta_, y, test_size=0.20, random_state=42)

In [5]:
import xgboost as xgb

In [41]:
xgb_model = xgb.XGBRegressor(learning_rate =0.1,
n_estimators=10000,
max_depth=3,
min_child_weight=3.0,
gamma=0.65,
reg_alpha=0.68,
subsample=0.75,
colsample_bytree=0.95,
objective= 'reg:linear',
nthread=6,
#scale_pos_weight=8.9,
seed=27, tree_method='gpu_hist',bins=64)

In [42]:
xgb_model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=50)

[0]	validation_0-rmse:0.412805
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.38494
[2]	validation_0-rmse:0.36077
[3]	validation_0-rmse:0.339925
[4]	validation_0-rmse:0.322021
[5]	validation_0-rmse:0.306746
[6]	validation_0-rmse:0.293715
[7]	validation_0-rmse:0.282707
[8]	validation_0-rmse:0.273476
[9]	validation_0-rmse:0.265729
[10]	validation_0-rmse:0.259258
[11]	validation_0-rmse:0.253816
[12]	validation_0-rmse:0.249328
[13]	validation_0-rmse:0.245585
[14]	validation_0-rmse:0.242525
[15]	validation_0-rmse:0.239952
[16]	validation_0-rmse:0.237808
[17]	validation_0-rmse:0.236053
[18]	validation_0-rmse:0.234612
[19]	validation_0-rmse:0.2334
[20]	validation_0-rmse:0.232367
[21]	validation_0-rmse:0.231516
[22]	validation_0-rmse:0.230794
[23]	validation_0-rmse:0.230246
[24]	validation_0-rmse:0.229739
[25]	validation_0-rmse:0.229298
[26]	validation_0-rmse:0.228961
[27]	validation_0-rmse:0.22869
[28]	validation_0-rmse:0.228427
[29]	validation_0-rmse

[251]	validation_0-rmse:0.223789
[252]	validation_0-rmse:0.223784
[253]	validation_0-rmse:0.22377
[254]	validation_0-rmse:0.223763
[255]	validation_0-rmse:0.223757
[256]	validation_0-rmse:0.223751
[257]	validation_0-rmse:0.223742
[258]	validation_0-rmse:0.223741
[259]	validation_0-rmse:0.223739
[260]	validation_0-rmse:0.223729
[261]	validation_0-rmse:0.223721
[262]	validation_0-rmse:0.223717
[263]	validation_0-rmse:0.223715
[264]	validation_0-rmse:0.223712
[265]	validation_0-rmse:0.223693
[266]	validation_0-rmse:0.223692
[267]	validation_0-rmse:0.223686
[268]	validation_0-rmse:0.223676
[269]	validation_0-rmse:0.223674
[270]	validation_0-rmse:0.223671
[271]	validation_0-rmse:0.223667
[272]	validation_0-rmse:0.223662
[273]	validation_0-rmse:0.223654
[274]	validation_0-rmse:0.223645
[275]	validation_0-rmse:0.22364
[276]	validation_0-rmse:0.223636
[277]	validation_0-rmse:0.223631
[278]	validation_0-rmse:0.223617
[279]	validation_0-rmse:0.223605
[280]	validation_0-rmse:0.223599
[281]	valida

[501]	validation_0-rmse:0.222672
[502]	validation_0-rmse:0.222667
[503]	validation_0-rmse:0.222661
[504]	validation_0-rmse:0.222662
[505]	validation_0-rmse:0.222657
[506]	validation_0-rmse:0.222654
[507]	validation_0-rmse:0.222651
[508]	validation_0-rmse:0.222649
[509]	validation_0-rmse:0.222647
[510]	validation_0-rmse:0.222643
[511]	validation_0-rmse:0.222642
[512]	validation_0-rmse:0.222641
[513]	validation_0-rmse:0.22264
[514]	validation_0-rmse:0.222638
[515]	validation_0-rmse:0.222631
[516]	validation_0-rmse:0.222626
[517]	validation_0-rmse:0.222624
[518]	validation_0-rmse:0.222621
[519]	validation_0-rmse:0.222619
[520]	validation_0-rmse:0.222614
[521]	validation_0-rmse:0.222608
[522]	validation_0-rmse:0.222607
[523]	validation_0-rmse:0.222608
[524]	validation_0-rmse:0.222606
[525]	validation_0-rmse:0.222604
[526]	validation_0-rmse:0.222603
[527]	validation_0-rmse:0.222602
[528]	validation_0-rmse:0.222599
[529]	validation_0-rmse:0.222595
[530]	validation_0-rmse:0.222591
[531]	valid

[750]	validation_0-rmse:0.22205
[751]	validation_0-rmse:0.222049
[752]	validation_0-rmse:0.222046
[753]	validation_0-rmse:0.222046
[754]	validation_0-rmse:0.22204
[755]	validation_0-rmse:0.222037
[756]	validation_0-rmse:0.222035
[757]	validation_0-rmse:0.222034
[758]	validation_0-rmse:0.222034
[759]	validation_0-rmse:0.222031
[760]	validation_0-rmse:0.222031
[761]	validation_0-rmse:0.222029
[762]	validation_0-rmse:0.222023
[763]	validation_0-rmse:0.22202
[764]	validation_0-rmse:0.222017
[765]	validation_0-rmse:0.222016
[766]	validation_0-rmse:0.222012
[767]	validation_0-rmse:0.222011
[768]	validation_0-rmse:0.22201
[769]	validation_0-rmse:0.22201
[770]	validation_0-rmse:0.222009
[771]	validation_0-rmse:0.222008
[772]	validation_0-rmse:0.222008
[773]	validation_0-rmse:0.222007
[774]	validation_0-rmse:0.222006
[775]	validation_0-rmse:0.222001
[776]	validation_0-rmse:0.221999
[777]	validation_0-rmse:0.221997
[778]	validation_0-rmse:0.221995
[779]	validation_0-rmse:0.221995
[780]	validatio

[999]	validation_0-rmse:0.2216
[1000]	validation_0-rmse:0.221598
[1001]	validation_0-rmse:0.221595
[1002]	validation_0-rmse:0.221592
[1003]	validation_0-rmse:0.22159
[1004]	validation_0-rmse:0.221589
[1005]	validation_0-rmse:0.221586
[1006]	validation_0-rmse:0.221586
[1007]	validation_0-rmse:0.221582
[1008]	validation_0-rmse:0.221582
[1009]	validation_0-rmse:0.221582
[1010]	validation_0-rmse:0.221579
[1011]	validation_0-rmse:0.22158
[1012]	validation_0-rmse:0.221579
[1013]	validation_0-rmse:0.221577
[1014]	validation_0-rmse:0.221575
[1015]	validation_0-rmse:0.221575
[1016]	validation_0-rmse:0.221575
[1017]	validation_0-rmse:0.221573
[1018]	validation_0-rmse:0.221572
[1019]	validation_0-rmse:0.22157
[1020]	validation_0-rmse:0.221569
[1021]	validation_0-rmse:0.221569
[1022]	validation_0-rmse:0.221569
[1023]	validation_0-rmse:0.221567
[1024]	validation_0-rmse:0.221565
[1025]	validation_0-rmse:0.221562
[1026]	validation_0-rmse:0.22156
[1027]	validation_0-rmse:0.221558
[1028]	validation_0-r

...

[4147]	validation_0-rmse:0.219833
[4148]	validation_0-rmse:0.219833
[4149]	validation_0-rmse:0.219833
[4150]	validation_0-rmse:0.219833
[4151]	validation_0-rmse:0.219832
[4152]	validation_0-rmse:0.219833
[4153]	validation_0-rmse:0.219831
[4154]	validation_0-rmse:0.219831
[4155]	validation_0-rmse:0.219831
[4156]	validation_0-rmse:0.219831
[4157]	validation_0-rmse:0.21983
[4158]	validation_0-rmse:0.21983
[4159]	validation_0-rmse:0.21983
[4160]	validation_0-rmse:0.21983
[4161]	validation_0-rmse:0.219828
[4162]	validation_0-rmse:0.219828
[4163]	validation_0-rmse:0.219827
[4164]	validation_0-rmse:0.219828
[4165]	validation_0-rmse:0.219827
[4166]	validation_0-rmse:0.219827
[4167]	validation_0-rmse:0.219827
[4168]	validation_0-rmse:0.219827
[4169]	validation_0-rmse:0.219827
[4170]	validation_0-rmse:0.219826
[4171]	validation_0-rmse:0.219826
[4172]	validation_0-rmse:0.219826
[4173]	validation_0-rmse:0.219826
[4174]	validation_0-rmse:0.219826
[4175]	validation_0-rmse:0.219825
[4176]	validation_

[4390]	validation_0-rmse:0.21978
[4391]	validation_0-rmse:0.21978
[4392]	validation_0-rmse:0.219781
[4393]	validation_0-rmse:0.219781
[4394]	validation_0-rmse:0.21978
[4395]	validation_0-rmse:0.21978
[4396]	validation_0-rmse:0.21978
[4397]	validation_0-rmse:0.21978
[4398]	validation_0-rmse:0.21978
[4399]	validation_0-rmse:0.21978
[4400]	validation_0-rmse:0.21978
[4401]	validation_0-rmse:0.21978
[4402]	validation_0-rmse:0.21978
[4403]	validation_0-rmse:0.21978
[4404]	validation_0-rmse:0.21978
[4405]	validation_0-rmse:0.21978
[4406]	validation_0-rmse:0.21978
[4407]	validation_0-rmse:0.21978
[4408]	validation_0-rmse:0.219779
[4409]	validation_0-rmse:0.219779
[4410]	validation_0-rmse:0.219779
[4411]	validation_0-rmse:0.219779
[4412]	validation_0-rmse:0.219778
[4413]	validation_0-rmse:0.219778
[4414]	validation_0-rmse:0.219777
[4415]	validation_0-rmse:0.219776
[4416]	validation_0-rmse:0.219776
[4417]	validation_0-rmse:0.219776
[4418]	validation_0-rmse:0.219776
[4419]	validation_0-rmse:0.219

[4633]	validation_0-rmse:0.219744
[4634]	validation_0-rmse:0.219744
[4635]	validation_0-rmse:0.219744
[4636]	validation_0-rmse:0.219744
[4637]	validation_0-rmse:0.219743
[4638]	validation_0-rmse:0.219743
[4639]	validation_0-rmse:0.219743
[4640]	validation_0-rmse:0.219742
[4641]	validation_0-rmse:0.219742
[4642]	validation_0-rmse:0.219741
[4643]	validation_0-rmse:0.219741
[4644]	validation_0-rmse:0.219741
[4645]	validation_0-rmse:0.21974
[4646]	validation_0-rmse:0.21974
[4647]	validation_0-rmse:0.21974
[4648]	validation_0-rmse:0.21974
[4649]	validation_0-rmse:0.21974
[4650]	validation_0-rmse:0.21974
[4651]	validation_0-rmse:0.21974
[4652]	validation_0-rmse:0.21974
[4653]	validation_0-rmse:0.21974
[4654]	validation_0-rmse:0.219739
[4655]	validation_0-rmse:0.219739
[4656]	validation_0-rmse:0.21974
[4657]	validation_0-rmse:0.219739
[4658]	validation_0-rmse:0.219739
[4659]	validation_0-rmse:0.21974
[4660]	validation_0-rmse:0.21974
[4661]	validation_0-rmse:0.21974
[4662]	validation_0-rmse:0.

[4876]	validation_0-rmse:0.219699
[4877]	validation_0-rmse:0.2197
[4878]	validation_0-rmse:0.2197
[4879]	validation_0-rmse:0.219701
[4880]	validation_0-rmse:0.219701
[4881]	validation_0-rmse:0.219701
[4882]	validation_0-rmse:0.219701
[4883]	validation_0-rmse:0.219701
[4884]	validation_0-rmse:0.219701
[4885]	validation_0-rmse:0.219701
[4886]	validation_0-rmse:0.2197
[4887]	validation_0-rmse:0.2197
[4888]	validation_0-rmse:0.2197
[4889]	validation_0-rmse:0.2197
[4890]	validation_0-rmse:0.219701
[4891]	validation_0-rmse:0.2197
[4892]	validation_0-rmse:0.219701
[4893]	validation_0-rmse:0.2197
[4894]	validation_0-rmse:0.2197
[4895]	validation_0-rmse:0.2197
[4896]	validation_0-rmse:0.2197
[4897]	validation_0-rmse:0.219701
[4898]	validation_0-rmse:0.219701
[4899]	validation_0-rmse:0.219701
[4900]	validation_0-rmse:0.219701
[4901]	validation_0-rmse:0.219701
[4902]	validation_0-rmse:0.219701
[4903]	validation_0-rmse:0.2197
[4904]	validation_0-rmse:0.2197
[4905]	validation_0-rmse:0.219701
[4906]

XGBRegressor(base_score=0.5, bins=64, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.95, gamma=0.65, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3.0, missing=None,
       n_estimators=10000, n_jobs=1, nthread=6, objective='reg:linear',
       random_state=0, reg_alpha=0.68, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.75, tree_method='gpu_hist')

In [ ]:
[998]	validation_0-rmse:0.221601
[999]	validation_0-rmse:0.2216

In [45]:
xgb_model = xgb.XGBRegressor(learning_rate =0.1,
n_estimators=10000,
max_depth=5,
min_child_weight=3.0,
gamma=0.65,
reg_alpha=0.68,
subsample=0.75,
colsample_bytree=0.95,
objective= 'reg:linear',
nthread=6,
#scale_pos_weight=8.9,
seed=27, tree_method='gpu_hist',bins=64)

In [46]:
xgb_model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=50)

[0]	validation_0-rmse:0.412572
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.384502
[2]	validation_0-rmse:0.360143
[3]	validation_0-rmse:0.339111
[4]	validation_0-rmse:0.321052
[5]	validation_0-rmse:0.305634
[6]	validation_0-rmse:0.292481
[7]	validation_0-rmse:0.281345
[8]	validation_0-rmse:0.271973
[9]	validation_0-rmse:0.264108
[10]	validation_0-rmse:0.257529
[11]	validation_0-rmse:0.25208
[12]	validation_0-rmse:0.247537
[13]	validation_0-rmse:0.243783
[14]	validation_0-rmse:0.240693
[15]	validation_0-rmse:0.238126
[16]	validation_0-rmse:0.235973
[17]	validation_0-rmse:0.234191
[18]	validation_0-rmse:0.232677
[19]	validation_0-rmse:0.231423
[20]	validation_0-rmse:0.230443
[21]	validation_0-rmse:0.229599
[22]	validation_0-rmse:0.228917
[23]	validation_0-rmse:0.228343
[24]	validation_0-rmse:0.22784
[25]	validation_0-rmse:0.227391
[26]	validation_0-rmse:0.227007
[27]	validation_0-rmse:0.226658
[28]	validation_0-rmse:0.226394
[29]	validation_0-r

[251]	validation_0-rmse:0.221565
[252]	validation_0-rmse:0.221561
[253]	validation_0-rmse:0.221557
[254]	validation_0-rmse:0.221547
[255]	validation_0-rmse:0.221545
[256]	validation_0-rmse:0.221539
[257]	validation_0-rmse:0.221532
[258]	validation_0-rmse:0.22153
[259]	validation_0-rmse:0.221521
[260]	validation_0-rmse:0.22152
[261]	validation_0-rmse:0.221512
[262]	validation_0-rmse:0.221501
[263]	validation_0-rmse:0.221493
[264]	validation_0-rmse:0.221488
[265]	validation_0-rmse:0.221481
[266]	validation_0-rmse:0.221473
[267]	validation_0-rmse:0.22147
[268]	validation_0-rmse:0.221464
[269]	validation_0-rmse:0.221458
[270]	validation_0-rmse:0.221455
[271]	validation_0-rmse:0.221441
[272]	validation_0-rmse:0.221441
[273]	validation_0-rmse:0.221438
[274]	validation_0-rmse:0.221418
[275]	validation_0-rmse:0.221404
[276]	validation_0-rmse:0.221399
[277]	validation_0-rmse:0.221395
[278]	validation_0-rmse:0.221394
[279]	validation_0-rmse:0.221388
[280]	validation_0-rmse:0.221381
[281]	validat

[500]	validation_0-rmse:0.22041
[501]	validation_0-rmse:0.220407
[502]	validation_0-rmse:0.220406
[503]	validation_0-rmse:0.220404
[504]	validation_0-rmse:0.2204
[505]	validation_0-rmse:0.220394
[506]	validation_0-rmse:0.220386
[507]	validation_0-rmse:0.220386
[508]	validation_0-rmse:0.220383
[509]	validation_0-rmse:0.220379
[510]	validation_0-rmse:0.220377
[511]	validation_0-rmse:0.220375
[512]	validation_0-rmse:0.220369
[513]	validation_0-rmse:0.220359
[514]	validation_0-rmse:0.220357
[515]	validation_0-rmse:0.220357
[516]	validation_0-rmse:0.220355
[517]	validation_0-rmse:0.220353
[518]	validation_0-rmse:0.220349
[519]	validation_0-rmse:0.220343
[520]	validation_0-rmse:0.220339
[521]	validation_0-rmse:0.22034
[522]	validation_0-rmse:0.220336
[523]	validation_0-rmse:0.220334
[524]	validation_0-rmse:0.220332
[525]	validation_0-rmse:0.220333
[526]	validation_0-rmse:0.220331
[527]	validation_0-rmse:0.22033
[528]	validation_0-rmse:0.220329
[529]	validation_0-rmse:0.220323
[530]	validatio

[750]	validation_0-rmse:0.219863
[751]	validation_0-rmse:0.219862
[752]	validation_0-rmse:0.219862
[753]	validation_0-rmse:0.219857
[754]	validation_0-rmse:0.219856
[755]	validation_0-rmse:0.219855
[756]	validation_0-rmse:0.21985
[757]	validation_0-rmse:0.219849
[758]	validation_0-rmse:0.219848
[759]	validation_0-rmse:0.219844
[760]	validation_0-rmse:0.219841
[761]	validation_0-rmse:0.219841
[762]	validation_0-rmse:0.219838
[763]	validation_0-rmse:0.219838
[764]	validation_0-rmse:0.219838
[765]	validation_0-rmse:0.219837
[766]	validation_0-rmse:0.219833
[767]	validation_0-rmse:0.219832
[768]	validation_0-rmse:0.219833
[769]	validation_0-rmse:0.219834
[770]	validation_0-rmse:0.219835
[771]	validation_0-rmse:0.219834
[772]	validation_0-rmse:0.219833
[773]	validation_0-rmse:0.219829
[774]	validation_0-rmse:0.219826
[775]	validation_0-rmse:0.219825
[776]	validation_0-rmse:0.219825
[777]	validation_0-rmse:0.219824
[778]	validation_0-rmse:0.219826
[779]	validation_0-rmse:0.219824
[780]	valid

[999]	validation_0-rmse:0.21952
[1000]	validation_0-rmse:0.219516
[1001]	validation_0-rmse:0.219513
[1002]	validation_0-rmse:0.219509
[1003]	validation_0-rmse:0.219508
[1004]	validation_0-rmse:0.219508
[1005]	validation_0-rmse:0.219508
[1006]	validation_0-rmse:0.219508
[1007]	validation_0-rmse:0.219509
[1008]	validation_0-rmse:0.219507
[1009]	validation_0-rmse:0.219504
[1010]	validation_0-rmse:0.219503
[1011]	validation_0-rmse:0.219503
[1012]	validation_0-rmse:0.219504
[1013]	validation_0-rmse:0.219502
[1014]	validation_0-rmse:0.219502
[1015]	validation_0-rmse:0.219502
[1016]	validation_0-rmse:0.219502
[1017]	validation_0-rmse:0.219501
[1018]	validation_0-rmse:0.219501
[1019]	validation_0-rmse:0.219502
[1020]	validation_0-rmse:0.219502
[1021]	validation_0-rmse:0.2195
[1022]	validation_0-rmse:0.219498
[1023]	validation_0-rmse:0.219497
[1024]	validation_0-rmse:0.219494
[1025]	validation_0-rmse:0.219493
[1026]	validation_0-rmse:0.219493
[1027]	validation_0-rmse:0.219493
[1028]	validation_

[1241]	validation_0-rmse:0.21933
[1242]	validation_0-rmse:0.21933
[1243]	validation_0-rmse:0.219329
[1244]	validation_0-rmse:0.219329
[1245]	validation_0-rmse:0.219329
[1246]	validation_0-rmse:0.219329
[1247]	validation_0-rmse:0.219326
[1248]	validation_0-rmse:0.219326
[1249]	validation_0-rmse:0.219325
[1250]	validation_0-rmse:0.219324
[1251]	validation_0-rmse:0.219322
[1252]	validation_0-rmse:0.219321
[1253]	validation_0-rmse:0.219322
[1254]	validation_0-rmse:0.21932
[1255]	validation_0-rmse:0.219317
[1256]	validation_0-rmse:0.219315
[1257]	validation_0-rmse:0.219316
[1258]	validation_0-rmse:0.219317
[1259]	validation_0-rmse:0.219316
[1260]	validation_0-rmse:0.219317
[1261]	validation_0-rmse:0.219316
[1262]	validation_0-rmse:0.219317
[1263]	validation_0-rmse:0.219316
[1264]	validation_0-rmse:0.219314
[1265]	validation_0-rmse:0.219315
[1266]	validation_0-rmse:0.219314
[1267]	validation_0-rmse:0.219313
[1268]	validation_0-rmse:0.219314
[1269]	validation_0-rmse:0.219313
[1270]	validation

[1483]	validation_0-rmse:0.2192
[1484]	validation_0-rmse:0.219201
[1485]	validation_0-rmse:0.219198
[1486]	validation_0-rmse:0.219196
[1487]	validation_0-rmse:0.219196
[1488]	validation_0-rmse:0.219196
[1489]	validation_0-rmse:0.219196
[1490]	validation_0-rmse:0.219195
[1491]	validation_0-rmse:0.219192
[1492]	validation_0-rmse:0.219193
[1493]	validation_0-rmse:0.219192
[1494]	validation_0-rmse:0.219191
[1495]	validation_0-rmse:0.219192
[1496]	validation_0-rmse:0.219192
[1497]	validation_0-rmse:0.219188
[1498]	validation_0-rmse:0.219189
[1499]	validation_0-rmse:0.219188
[1500]	validation_0-rmse:0.219188
[1501]	validation_0-rmse:0.219188
[1502]	validation_0-rmse:0.219188
[1503]	validation_0-rmse:0.219188
[1504]	validation_0-rmse:0.219188
[1505]	validation_0-rmse:0.219187
[1506]	validation_0-rmse:0.219185
[1507]	validation_0-rmse:0.219185
[1508]	validation_0-rmse:0.219185
[1509]	validation_0-rmse:0.219184
[1510]	validation_0-rmse:0.219184
[1511]	validation_0-rmse:0.219186
[1512]	validatio

[1725]	validation_0-rmse:0.219122
[1726]	validation_0-rmse:0.219121
[1727]	validation_0-rmse:0.219121
[1728]	validation_0-rmse:0.21912
[1729]	validation_0-rmse:0.21912
[1730]	validation_0-rmse:0.219121
[1731]	validation_0-rmse:0.21912
[1732]	validation_0-rmse:0.21912
[1733]	validation_0-rmse:0.219118
[1734]	validation_0-rmse:0.219119
[1735]	validation_0-rmse:0.219117
[1736]	validation_0-rmse:0.219119
[1737]	validation_0-rmse:0.219118
[1738]	validation_0-rmse:0.219118
[1739]	validation_0-rmse:0.219117
[1740]	validation_0-rmse:0.219117
[1741]	validation_0-rmse:0.219118
[1742]	validation_0-rmse:0.219116
[1743]	validation_0-rmse:0.219116
[1744]	validation_0-rmse:0.219114
[1745]	validation_0-rmse:0.219115
[1746]	validation_0-rmse:0.219115
[1747]	validation_0-rmse:0.219113
[1748]	validation_0-rmse:0.219114
[1749]	validation_0-rmse:0.219114
[1750]	validation_0-rmse:0.219115
[1751]	validation_0-rmse:0.219113
[1752]	validation_0-rmse:0.219113
[1753]	validation_0-rmse:0.219113
[1754]	validation_

[1967]	validation_0-rmse:0.219067
[1968]	validation_0-rmse:0.219067
[1969]	validation_0-rmse:0.219069
[1970]	validation_0-rmse:0.219069
[1971]	validation_0-rmse:0.219067
[1972]	validation_0-rmse:0.219067
[1973]	validation_0-rmse:0.219067
[1974]	validation_0-rmse:0.219067
[1975]	validation_0-rmse:0.219066
[1976]	validation_0-rmse:0.219067
[1977]	validation_0-rmse:0.219067
[1978]	validation_0-rmse:0.219065
[1979]	validation_0-rmse:0.219064
[1980]	validation_0-rmse:0.219065
[1981]	validation_0-rmse:0.219065
[1982]	validation_0-rmse:0.219064
[1983]	validation_0-rmse:0.219064
[1984]	validation_0-rmse:0.219065
[1985]	validation_0-rmse:0.219064
[1986]	validation_0-rmse:0.219064
[1987]	validation_0-rmse:0.219063
[1988]	validation_0-rmse:0.219064
[1989]	validation_0-rmse:0.219066
[1990]	validation_0-rmse:0.219066
[1991]	validation_0-rmse:0.219068
[1992]	validation_0-rmse:0.219069
[1993]	validation_0-rmse:0.219068
[1994]	validation_0-rmse:0.219069
[1995]	validation_0-rmse:0.219069
[1996]	validat

XGBRegressor(base_score=0.5, bins=64, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.95, gamma=0.65, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=3.0, missing=None,
       n_estimators=10000, n_jobs=1, nthread=6, objective='reg:linear',
       random_state=0, reg_alpha=0.68, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.75, tree_method='gpu_hist')

In [61]:
xgb_model = xgb.XGBRegressor(learning_rate =0.1,
n_estimators=10000,
max_depth=7,
min_child_weight=3.0,
gamma=0.65,
reg_alpha=0.68,
subsample=0.75,
colsample_bytree=0.95,
objective= 'reg:linear',
nthread=6,
#scale_pos_weight=8.9,
seed=27, tree_method='gpu_hist',bins=64)


In [62]:
xgb_model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=50)

[0]	validation_0-rmse:0.412412
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.384178
[2]	validation_0-rmse:0.359667
[3]	validation_0-rmse:0.338497
[4]	validation_0-rmse:0.320308
[5]	validation_0-rmse:0.304729
[6]	validation_0-rmse:0.291497
[7]	validation_0-rmse:0.280271
[8]	validation_0-rmse:0.270797
[9]	validation_0-rmse:0.262868
[10]	validation_0-rmse:0.256232
[11]	validation_0-rmse:0.250721
[12]	validation_0-rmse:0.24612
[13]	validation_0-rmse:0.242294
[14]	validation_0-rmse:0.239155
[15]	validation_0-rmse:0.236544
[16]	validation_0-rmse:0.234401
[17]	validation_0-rmse:0.232614
[18]	validation_0-rmse:0.231156
[19]	validation_0-rmse:0.229941
[20]	validation_0-rmse:0.228899
[21]	validation_0-rmse:0.227993
[22]	validation_0-rmse:0.227274
[23]	validation_0-rmse:0.226691
[24]	validation_0-rmse:0.226179
[25]	validation_0-rmse:0.225805
[26]	validation_0-rmse:0.225344
[27]	validation_0-rmse:0.225026
[28]	validation_0-rmse:0.224716
[29]	validation_0-

KeyboardInterrupt: 

In [6]:
import gc

In [17]:
gc.collect()

72

In [6]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
dtest = xgb.DMatrix(X_test_full)

In [15]:


# set xgboost params
param = {'learning_rate':0.1,
'n_estimators':10000,
'max_depth':5,
'min_child_weight':3.0,
'gamma':0.75,
'reg_alpha':0.68,
'reg_lambda': 0.6,
'subsample':0.75,
'colsample_bytree':0.85,
"objective":"reg:linear",
'nthread':6,
#scale_pos_weight=8.9,
'seed':27, 'tree_method':'gpu_hist','bins':64}  # the number of classes that exist in this datset
num_round = 2000  # the number of training iterations

#------------- numpy array ------------------
# training and testing - numpy matrices
#bst = xgb.train(param, dtrain, num_round)
#preds = bst.predict(dtest)

In [10]:
param = {'learning_rate':0.1,
'n_estimators':10000,
'max_depth':5,
'min_child_weight':3.0,
'gamma':0.75,
'reg_alpha':0.68,
'reg_lambda': 0.6,
'subsample':0.75,
'colsample_bytree':0.85,
"objective":"reg:linear",
'nthread':6,
#scale_pos_weight=8.9,
'seed':27, 'tree_method':'gpu_hist','bins':64}
# [2200]	train-rmse:0.208278	valid-rmse:0.218938
# Stopping. Best iteration:
# [2172]	train-rmse:0.208406	valid-rmse:0.218934

In [16]:
bst = xgb.train(param, dtrain, 3000, evals=([(dtrain, "train"),(dvalid, "valid")]),
                verbose_eval=100, early_stopping_rounds=50)

[0]	train-rmse:0.412611	valid-rmse:0.412569
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.222596	valid-rmse:0.223059
[200]	train-rmse:0.220868	valid-rmse:0.221853
[300]	train-rmse:0.219661	valid-rmse:0.221175
[400]	train-rmse:0.218659	valid-rmse:0.220693
[500]	train-rmse:0.217798	valid-rmse:0.220369
[600]	train-rmse:0.217014	valid-rmse:0.220114
[700]	train-rmse:0.216307	valid-rmse:0.219913
[800]	train-rmse:0.215653	valid-rmse:0.219763
[900]	train-rmse:0.214996	valid-rmse:0.219608
[1000]	train-rmse:0.214396	valid-rmse:0.219486
[1100]	train-rmse:0.213829	valid-rmse:0.219384
[1200]	train-rmse:0.21327	valid-rmse:0.21931
[1300]	train-rmse:0.21272	valid-rmse:0.219238
[1400]	train-rmse:0.212203	valid-rmse:0.219184
[1500]	train-rmse:0.211671	valid-rmse:0.219153
[1600]	train-rmse:0.211169	valid-rmse:0.219123
[1700]	train-rmse:0.210693	valid-rmse:0.219087
[1800]	train-rmse:0.2101

In [11]:
bst = xgb.train(param, dtrain, 3000, evals=([(dtrain, "train"),(dvalid, "valid")]),
                verbose_eval=100, early_stopping_rounds=50)

[0]	train-rmse:0.412611	valid-rmse:0.412569
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.222643	valid-rmse:0.223109
[200]	train-rmse:0.220915	valid-rmse:0.221906
[300]	train-rmse:0.219664	valid-rmse:0.221205
[400]	train-rmse:0.218645	valid-rmse:0.220742
[500]	train-rmse:0.217747	valid-rmse:0.220387
[600]	train-rmse:0.216982	valid-rmse:0.22013
[700]	train-rmse:0.216257	valid-rmse:0.219923
[800]	train-rmse:0.215621	valid-rmse:0.219782
[900]	train-rmse:0.21499	valid-rmse:0.219644
[1000]	train-rmse:0.214405	valid-rmse:0.219558
[1100]	train-rmse:0.213806	valid-rmse:0.219471
[1200]	train-rmse:0.213245	valid-rmse:0.219405
[1300]	train-rmse:0.212688	valid-rmse:0.219322
[1400]	train-rmse:0.212144	valid-rmse:0.219271
[1500]	train-rmse:0.211622	valid-rmse:0.21924
[1600]	train-rmse:0.211111	valid-rmse:0.2192
[1700]	train-rmse:0.210603	valid-rmse:0.219166
[1800]	train-rmse:0.210118

In [2]:
param = {'learning_rate':0.1,
'n_estimators':10000,
'max_depth':3,
'min_child_weight':3.0,
'gamma':0.75,
'reg_alpha':0.68,
'reg_lambda': 0.6,
'subsample':0.75,
'colsample_bytree':0.85,
"objective":"reg:linear",
'nthread':6,
#scale_pos_weight=8.9,
'seed':27, 'tree_method':'gpu_hist','bins':64}
# [2200]	train-rmse:0.208278	valid-rmse:0.218938
# Stopping. Best iteration:
# [2172]	train-rmse:0.208406	valid-rmse:0.218934

In [19]:
bst.__del__()

In [9]:
bst = xgb.train(param, dtrain, 3000, evals=([(dtrain, "train"),(dvalid, "valid")]),
                verbose_eval=100, early_stopping_rounds=50)

[0]	train-rmse:0.412834	valid-rmse:0.412805
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.225242	valid-rmse:0.225274
[200]	train-rmse:0.22407	valid-rmse:0.224186
[300]	train-rmse:0.223354	valid-rmse:0.223549
[400]	train-rmse:0.222829	valid-rmse:0.223109
[500]	train-rmse:0.22239	valid-rmse:0.222756
[600]	train-rmse:0.221994	valid-rmse:0.222438
[700]	train-rmse:0.221702	valid-rmse:0.222219
[800]	train-rmse:0.221388	valid-rmse:0.221987
[900]	train-rmse:0.221099	valid-rmse:0.221783
[1000]	train-rmse:0.220875	valid-rmse:0.221643
[1100]	train-rmse:0.220643	valid-rmse:0.221479
[1200]	train-rmse:0.220432	valid-rmse:0.221359
[1300]	train-rmse:0.220223	valid-rmse:0.221223
[1400]	train-rmse:0.220037	valid-rmse:0.221119
[1500]	train-rmse:0.219862	valid-rmse:0.22102
[1600]	train-rmse:0.21969	valid-rmse:0.220938
[1700]	train-rmse:0.21954	valid-rmse:0.220876
[1800]	train-rmse:0.219381

In [7]:
1

1

In [6]:
d = pd.read_csv('train.csv')
y = d['deal_probability']
del d
X_meta_ = np.load('X_train_xgb.npy')
X_test_full = np.load('X_test_xgb.npy')

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X_meta_, y, test_size=0.20, random_state=42)
params = {'learning_rate': 0.05, 'max_depth': 3, 'boosting': 'gbdt', 'objective': 'regression',
          'metric': ['logloss', 'rmse'],
          'is_training_metric': True, 'seed': 19, 'num_leaves': 256, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
          'bagging_freq': 5, 'lambda_l1': 4, 'lambda_l2': 4, 'num_threads': 12}
model = lgb.train(params, lgb.Dataset(X_train, label=y_train),10000,
                           lgb.Dataset(X_valid, label=y_valid),
                           verbose_eval=100, early_stopping_rounds=100)
print(rmse(model.predict(X_valid), y_valid))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.226497
[200]	valid_0's rmse: 0.225213
[300]	valid_0's rmse: 0.224634
[400]	valid_0's rmse: 0.224243
[500]	valid_0's rmse: 0.223899
[600]	valid_0's rmse: 0.223614
[700]	valid_0's rmse: 0.223319
[800]	valid_0's rmse: 0.223112
[900]	valid_0's rmse: 0.222925
[1000]	valid_0's rmse: 0.222756
[1100]	valid_0's rmse: 0.222598
[1200]	valid_0's rmse: 0.222468
[1300]	valid_0's rmse: 0.222358
[1400]	valid_0's rmse: 0.222236
[1500]	valid_0's rmse: 0.22212
[1600]	valid_0's rmse: 0.222021
[1700]	valid_0's rmse: 0.22192
[1800]	valid_0's rmse: 0.22183
[1900]	valid_0's rmse: 0.22175
[2000]	valid_0's rmse: 0.221657
[2100]	valid_0's rmse: 0.221566
[2200]	valid_0's rmse: 0.221503
[2300]	valid_0's rmse: 0.221435
[2400]	valid_0's rmse: 0.221375
[2500]	valid_0's rmse: 0.221315
[2600]	valid_0's rmse: 0.221243
[2700]	valid_0's rmse: 0.221182
[2800]	valid_0's rmse: 0.221111
[2900]	valid_0's rmse: 0.221058
[3000]	valid_0's rmse:

In [8]:
pred = model.predict(X_test_full)
val_pred = model.predict(X_valid)

In [9]:
np.mean(pred), np.mean(val_pred)

(0.1498356617361346, 0.13899634706967104)